# Chatbot

In this notebook, we study how to have extended conversations.

## Setup

In [5]:
import openai
import os

from IPython.display import display, HTML
from redlines import Redlines
from IPython.display import Markdown

from dotenv import load_dotenv
load_dotenv(".env")

openai.api_key = os.environ["OPENAI_API_KEY"]

In [6]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0) -> str:
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    return response.choices[0].message["content"]

In [4]:
messages = [
    {"role": "system", "content": "You are an assistant that speaks like Shakespeare"},
    {"role": "user", "content": "tell me a joke"},
    {"role": "assistant", "content": "Why did the chicken cross the road"},
    {"role": "user", "content": "I don't know"}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Verily, 'twas to get to the other side!


## No context

If we provide only one message in the request, the model has no context and cannot really do a conversation.

In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa, it's nice to meet you! How can I assist you today?


In [6]:
messages = [
    {"role": "system", "content": "You are friendly chatbot"},
    {"role": "user", "content": "Yes, can you remind me, what is my name?"}
]
print(get_completion_from_messages(messages, temperature=1))

I apologize, but as an AI language model, I do not have access to personal information like your name.


## Adding context

In [7]:
messages = [
    {"role": "system", "content": "You are friendly chatbot"},
    {"role": "user", "content": "Hi, my name is Isa"},
    {"role": "assistant", "content": "Hi Isa! It's nice to meet you. \
    Is there anything I can help you with today?"},
    {"role": "user", "content": "Yes, you can remind me, what is my name?"},
]
print(get_completion_from_messages(messages, temperature=1))
    

Your name is Isa 😊.


## My example

In [8]:
messages = [
    {"role": "system", "content": "You are a translator from Russian to English in a business setting"},
    {"role": "user", "content": "Я хотел бы встретиться с Вами в ближайшую среду"},
]
print(get_completion_from_messages(messages, temperature=1))

"I would like to meet with you next Wednesday."


## OrderBot

In [7]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({"role": "user", "content": f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({"role": "assistant", "content": f"{response}"})
    panels.append(
        pn.Row("User: ", pn.pane.Markdown(prompt, width=600))
    )
    panels.append(
         pn.Row("Assistant: ", pn.pane.Markdown(response, width=600, style={"background-color": "#F6F6F6"}))
    )
    
    return pn.Column(*panels)
                   

In [8]:
import panel as pn  # GUI
pn.extension()

panels = []  # collect display

context = [
    {"role": "system", "content": """
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""
    }
]

inp = pn.widgets.TextInput(value="Hi", placeholder="Enter text here...")
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

/var/folders/n9/k_tmh3lj3_33kn5rj5dzrqg00000gn/T/ipykernel_82782/188012497.py:11: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.1, use 'styles' instead.
  pn.Row("Assistant: ", pn.pane.Markdown(response, width=600, style={"background-color": "#F6F6F6"}))


Column
    [0] TextInput(placeholder='Enter text here...')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [10]:
messages = context.copy()
messages.append(
    {"role": "system", "content": "create a json summary of the previous food order. Itemize the price for each item \
    The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size 4) list of sides, include size 5) total price"},
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

Sure, here's a JSON summary of the previous food order:

```
{
  "pizza": {
    "type": "pepperoni",
    "size": "small",
    "toppings": [
      "extra cheese"
    ],
    "price": 14.95
  },
  "drinks": [
    {
      "type": "Coke",
      "size": "medium",
      "price": 2.00
    }
  ],
  "sides": [],
  "total_price": 16.95
}
```


In [11]:
messages

[{'role': 'system',
  'content': "\nYou are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 "},
 {'role': 'user', 'content': ''},
 {'role': 'assistant',
  'content': 'Hello! Welcome to 

As we can see from the conversation and the JSON output, the Artificial General Intelligence is not yet with us. First, the computed price for the pizza corresponds to the large size, although I have ordered a small one. Second, the total price in the conversation is wrong (\\$15.95), while in the JSON output is right (\\$16.95).